# Klasyfikacja obrazu za pomocą konwolucyjnych sieci neuronowych 

Wykorzystamy konwolucyjne sieci neuronowe (ang. convolutional neural networks - CNNs), by nauczyć nasz komputer jak widzieć - a jest to coś, co jest możliwe jedynie dzięki głębokiemu uczeniu (ang. deep learning).

## Wprowadzenie do zadania "Old Polish Cars"

Spróbujemy stworzyć model, który pozwoli rozróżnić dwie marki starych polskich samochodów. Dostępne jest 2400 oznakowanych zdjęć samochodów do trenowania i 270 w zestawie testowym, które musimy spróbować oznakować.

In [0]:
# Umieść je na górze każdego notatnika, aby uzyskać automatyczne ponowne ładowanie
%reload_ext autoreload
%autoreload 2

#matplotlib setup
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import matplotlib.style
import matplotlib as mpl
mpl.style.use('default')
mpl.style.use('seaborn-ticks')
# %config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import shutil as sh

Tutaj importujemy niezbędne biblioteki, do czego konkretnie które służą dowiemy się podczas kursu (http://course.fast.ai).

In [0]:
# Ten plik zawiera wszystkie główne biblioteki zewnętrzne, których użyjemy.
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

`PATH` jest ścieżką do twoich danych - jeśli wykorzystasz zalecaną metodę konfiguracji tej lekcji, nie będziesz musiał tego zmieniać. `sz` to rozmiar, do którego obrazy zostaną przeskalowane w celu uzyskania szybkich przebiegów treningowych. Podczas kursu będziemy sporo rozmawiać o tym parametrze. Zostaw na razie `224`.

In [0]:
PATH = "/content/data/old_polish_cars/old_polish_cars_two_classes_v2a-512-split/"
sz=224

Ważne jest, aby mieć działający GPU NVidii. Zestaw bibliotek stosowany za kulisami do pracy z procesorami graficznymi NVidii nosi nazwę CUDA. Dlatego zanim przejdzisz dalej, upewnij się, że następująca linia zwraca wartość `True`. Jeśli masz z tym problemy, sprawdź FAQ i poproś o pomoc [na forum](http://forums.fast.ai).

In [0]:
torch.cuda.is_available()

Ponadto NVidia zapewnia specjalne przyspieszone funkcje do głębokiego uczenia w pakiecie o nazwie CuDNN. Chociaż nie jest to bezwzględnie konieczne, znacznie poprawi wydajność treningów i jest domyślnie uwzględnione we wszystkich obsługiwanych konfiguracjach fastai. Dlatego jeśli poniższe nie zwróci wartości `True`, warto sprawdzić, dlaczego.

In [0]:
torch.backends.cudnn.enabled

## Najpierw obejrzyj zdjęcia samochodów

Biblioteka `fastai` zakłada, że posiadasz katalogi *train* i *valid*. Zakłada się także, że każdy katalog będzie zawierał podkatalogi dla każdej klasy, która ma zostać rozpaznana (w tym przypadku "maluch", "polonez" itp.).

In [0]:
os.listdir(PATH)

In [0]:
os.listdir(f'{PATH}valid')

In [0]:
files = os.listdir(f'{PATH}valid/maluch')[:5]
files

In [0]:
img = plt.imread(f'{PATH}valid/maluch/{files[2]}')
plt.imshow(img);

Oto jak wyglądają dane surowe.

In [0]:
img.shape

In [0]:
img[:4,:4]

## Nasz pierwszy model: szybki start


Użyjemy **wcześniej wytrenowanego** modelu, czyli modelu stworzonego przez kogoś innego dla rozwiązania innego problemu. Zamiast budować model od zera, aby rozwiązać podobny problem, jako punkt wyjścia wykorzystamy model wytrenowany na ImageNet (1,2 miliona obrazów i 1000 klas). Model ten to Convolutional Neural Network (CNN), rodzaj sieci neuronowej, którą używa się budując najnowocześniejsze modele rozpoznawania obrazów. Podczas tego kursu będziemy się uczyć wszystkiego o CNN.

Będziemy używać model **resnet34**. resnet34 to wersja modelu, który wygrał konkurs ImageNet 2015. Tutaj jest więcej informacji na temat [modeli resnet] (https://github.com/KaimingHe/deep-residual-networks). Przyjrzymy się im dogłębnie później, ale na razie skupimy się na ich efektywnym wykorzystaniu.

Oto jak wytrenować i zewaluować model *old polish cars* przy pomocy 3 linii kodu i w mniej niż 20 sekund:

In [0]:
# Odkomentuj poniższe informacje, jeśli chcesz zresetować swoje wstępne aktywacje
# shutil.rmtree(f'{PATH}tmp', ignore_errors=True)

In [0]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 3)

Jak dobry jest ten model? Kilka lat temu, przy podobnym zadaniu, osiągano maksymalnie 80% dokładności. Ale konkurs rozpoznawania obrazów ogłoszony przez Kaggle, spowodował ogromny skok do 98,9% dokładności, a autor pewnej popularnej biblioteki deep learning zwyciężył w tym konkursie.

## Analizowanie wyników: oglądanie zdjęć

Oprócz ogólnego spojrzenia na wskaźniki warto przyjrzeć się poniższym przykładom:

1. Kilka losowo wybranych poprawnych rozpoznań
2. Kilka losowo wybranych nieprawidłowych rozpoznań
3. Najbardziej poprawnie rozpoznania każdej klasy (tj. te z najwyższym prawdopodobieństwem z poprawnych)
4. Najbardziej niepoprawne rozpoznania każdej klasy (tj. te z najwyższym prawdopodobieństwem z nieprawidłowych)
5. Najbardziej niepewne rozpoznania (tj. te z prawdopodobieństwem najbliższym 0,5).

In [0]:
# To jest etykieta dla danych walidacyjnych
data.val_y

In [0]:
# stąd wiemy, że "maluch" to etykieta 0, a "polonez" to etykieta 1.
data.classes

In [0]:
# daje to przewidywanie dla zestawu walidacyjnego. Prognozy są w skali logarytmicznej
log_preds = learn.predict()
log_preds.shape

In [0]:
log_preds[:10]

In [0]:
preds = np.argmax(log_preds, axis=1)  # od logarytmu prawdopodobieństwa do 0 lub 1
probs = np.exp(log_preds[:,1])        # pr(polonez)

In [0]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds == data.val_y)==is_correct)

In [0]:
def plot_val_with_title(idxs, title):
    imgs = np.stack([data.val_ds[x][0] for x in idxs])
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(data.val_ds.denorm(imgs), rows=1, titles=title_probs)

In [0]:
def plots(ims, figsize=(12,6), rows=1, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

In [0]:
def load_img_id(ds, idx): return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

In [0]:
# 1. Kilka losowo wybranych poprawnych rozpoznań
plot_val_with_title(rand_by_correct(True), "Correctly classified")

In [0]:
# 2. Kilka losowo wybranych nieprawidłowych rozpoznań
plot_val_with_title(rand_by_correct(False), "Incorrectly classified")

In [0]:
def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)

In [0]:
plot_val_with_title(most_by_correct(0, True), "Most correct maluchs")

In [0]:
plot_val_with_title(most_by_correct(1, True), "Most correct polonezes")

In [0]:
plot_val_with_title(most_by_correct(0, False), "Most incorrect maluchs")

In [0]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect polonezes")

In [0]:
most_uncertain = np.argsort(np.abs(probs -0.5))[:4]
plot_val_with_title(most_uncertain, "Most uncertain predictions")

## Wybór tempa uczenia się

*Tempo uczenia się* określa, jak szybko lub jak wolno chcesz aktualizować *wagi* (lub *parametry*). Tempo uczenia się jest jednym z najtrudniejszych parametrów do ustawienia, ponieważ ma znaczący wpływ na wydajność modelu.

Metoda `learn.lr_find()` pomaga znaleźć optymalne tempo uczenia się. Wykorzystuje technikę opracowaną w 2015 roku, opisaną w publikacji [Cyclical Learning Rates for Training Neural Networks](http://arxiv.org/abs/1506.01186), gdzie po prostu ciągle zwiększamy tempo uczenia się, zaczynając od bardzo małej wartości, kończąc kiedy strata przestaje maleć. Możemy stworzyć wykres tempa uczenia się poprzez wszystkie porcje danych jednej epoki, aby zobaczyć, jak to wygląda.

Najpierw tworzymy nowego ucznia `learn`, ponieważ chcemy wiedzieć, jak ustawić tempo uczenia się dla nowego (niewytrenowanego) modelu.

In [0]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [0]:
lrf=learn.lr_find()

Nasz obiekt `learn` ma atrybut `sched`, który zawiera naszego 'planistę' szybkości uczenia się i ma kilka przydatnych wykresów, w tym:



In [0]:
mpl.style.use('bmh')
learn.sched.plot_lr()

Note that in the previous plot iteration is one iteration (or minibatch) of SGD. In one epoch there are (num_train_samples/num_iterations) of SGD.

Zauważ, że na poprzednim wykresie jedna iteracja to iteracja (lub 'mini-batch' algorytmu SGD). Jedna epoka składa się z (num_train_samples/num_iterations) SGD.

Widzimy wykres straty w stosunku do tempa uczenia się, aby zobaczyć, gdzie zmniejsza się nasza strata:

In [0]:
learn.sched.plot()

Strata wciąż wyraźnie maleje dla `lr=1e-2 (0.01)`, więc tego użyjemy. Zwróć uwagę, że optymalne tempo uczenia się może się zmieniać podczas treningu modelu, więc możesz chcieć ponownie uruchomić tę funkcję co jakiś czas.

## Polepszenie naszego modelu

### Augmentacja danych

Jeśli spróbujesz trenować przez więcej epok, zauważysz, że zaczynamy *przetrenowywać*, co oznacza, że nasz model uczy się rozpoznawać konkretne obrazy w zbiorze treningowym, zamiast generalizować tak, że uzyskamy dobre wyniki również na zestawie walidacyjnym. Jednym ze sposobów rozwiązania tego problemu jest faktyczne stworzenie większej ilości danych, dzięki *augmentacji danych*. Chodzi o losową modyfikację obrazów w sposób, który nie powinien wpływać na ich interpretację, takich jak poziome odbijanie, powiększanie i obracanie.

Możemy to zrobić, przekazując `aug_tfms` (*augmentation transforms*) do `tfms_from_model()`, z listą funkcji do zastosowania, które losowo zmieniają obraz, jak tylko chcemy. W przypadku zdjęć, które są w dużej mierze robione z boku (np. większość zdjęć samochodów, w przeciwieństwie do zdjęć zrobionych od góry, takich jak zdjęcia satelitarne), możemy użyć wstępnie zdefiniowanej listy funkcji `transforms_side_on`. Możemy również określić losowe powiększanie obrazów do określonej skali, dodając parametr `max_zoom`.

In [0]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [0]:
def get_augs():
    data = ImageClassifierData.from_paths(PATH, bs=2, tfms=tfms, num_workers=1)
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

In [0]:
ims = np.stack([get_augs() for i in range(6)])

In [0]:
plots(ims, rows=2)

Stwórzmy nowy obiekt `data`, który będzie mieć augmentacje w swoim zestawie transformacji.

In [0]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [0]:
learn.fit(1e-2, 1)

In [0]:
learn.precompute=False

Domyślnie, gdy tworzymy ucznia (`learn`), ustawia on wszystkie oprócz ostatniej warstwy na *zamrożone*. Oznacza to, że kiedy wywołujemy `fit()`, uczeń aktualizuje wagi tylko w ostatniej warstwie.

In [0]:
learn.fit(1e-2, 3, cycle_len=1)

Co to jest parametr `cycle_len`? To, co tutaj robimy, to metoda *stochastycznego gradientu prostego z restartem (ang. stochastic gradient descent with restarts - SGDR)*, jest to wariant *wyżarzania szybkości uczenia (ang. learning rate annealing)*, która stopniowo zmniejsza tempo uczenia się w miarę postępu treningu. To pomaga, bo gdy zbliżamy się do optymalnych wag, chcemy podejść tam mniejszymi krokami.

Jednakże możemy znaleźć się w części przestrzeni wagi, która nie jest bardzo odporna - to znaczy niewielkie zmiany wag mogą spowodować duże zmiany w stracie. Chcemy zachęcić nasz model do znalezienia części przestrzeni wag, które są zarówno dokładne, jak i stabilne. Dlatego od czasu do czasu zwiększamy szybkość uczenia się (jest to "restart" w "SGDR"), co zmusi model do przejścia do innej części przestrzeni wag, jeśli obecny obszar jest "spiczasty". Oto wykres tego, jak to może wyglądać, jeśli zresetujemy tempo uczenia się 3 razy (w tej publikacji nazywają to "cyklicznym harmonogramem LR", ang. 'cyclic LR schedule'):

![SGDR](https://raw.githubusercontent.com/fastai/fastai/master/courses/dl1/images/sgdr.png)
(Z publikacji [Snapshot Ensembles](https://arxiv.org/abs/1704.00109)).

Liczba epok pomiędzy resetowaniem tempa uczenia się jest ustalana przez `cycle_len`, a liczba razy, kiedy to się dzieje, jest określana jako *liczba cykli*, i jest tym, co faktycznie przekazujemy jako drugi parametr do `fit()`. Oto jak wyglądało nasze rzeczywiste tempo uczenia się:

In [0]:
learn.sched.plot_lr()

Nasza strata na zestawie walidacyjnym nie ulega znacznej poprawie, więc prawdopodobnie nie ma potrzeby dalszego trenowania samej ostatniej warstwy.

Ponieważ w tym momencie mamy już całkiem niezły model, możemy chcieć go zapisać, abyśmy mogli go załadować ponownie później bez trenowania od zera.

In [0]:
learn.save('224_lastlayer')

In [0]:
learn.load('224_lastlayer')

### Tuning i 'różnicowe hartowanie kroku uczenia się'

Teraz, gdy mamy już wytrenowaną dobrą warstwę końcową, możemy spróbować dopracować pozostałe warstwy. Aby powiedzieć uczniowi, że chcemy odblokować pozostałe warstwy, wystarczy użyć `unfreeze()`.

In [0]:
learn.unfreeze()

Zauważ, że pozostałe warstwy zostały * już * wytrenowane do rozpoznawania zdjęć imagenetowych (podczas gdy nasze końcowe warstwy zostały losowo zainicjowane), więc chcemy uważać, aby nie zniszczyć dokładnie wyregulowanych wag, które już tam są.

Ogólnie mówiąc, wcześniejsze warstwy (jak widzieliśmy) mają więcej funkcji ogólnego przeznaczenia. Dlatego oczekiwalibyśmy, że będą potrzebowali mniejszego dostrajania nowych zestawów danych. Z tego powodu użyjemy różnych poziomów nauki dla różnych warstw: pierwsze kilka warstw będzie na poziomie 1e-4, środkowe warstwy na 1e-3, a nasze warstwy FC opuścimy na 1e-2, tak jak poprzednio. Mówimy o tym jako o *zróżnicowanych tempach uczenia się*, chociaż nie ma standardowej nazwy dla tej technologii w literaturze, o której nam wiadomo.

In [0]:
lr=np.array([1e-4,1e-3,1e-2])

In [0]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

Kolejną sztuczką, której tutaj użyliśmy jest dodanie parametru `cycle_mult`. 

Cykle są mierzone w epokach, więc cycle_len = 1 samo w sobie oznaczałoby ciągłe zmniejszanie szybkości uczenia się w trakcie jednej epoki, a następnie przeskakiwanie z powrotem do góry. Parametr cycle_mult mówi, aby pomnożyć długość cyklu przez coś (w tym przypadku 2), jak tylko ukończysz jeden cykl.

In [0]:
learn.sched.plot_lr()

Zauważ, że to, co przedstawiono powyżej, to tempo uczenia się *ostatnich warstw*. Współczynniki uczenia się wcześniejszych warstw są ustalane na tych samych wielokrotnościach ostatecznych poziomów warstw, zgodnie z pierwotnym żądaniem (tj. Pierwsze warstwy mają 100x mniejsze, a warstwy średnie 10x mniejsze szybkości uczenia się, ponieważ ustawiamy `lr=np.array([1e-4,1e-3,1e-2])`.

In [0]:
learn.save('224_all')

In [0]:
learn.load('224_all')

Jest jeszcze coś, co możemy zrobić z rozszerzeniem danych: użyj go w *czasie wnioskowania* (znanym również jako * czas testu*). Nic dziwnego, że jest to znane jako *augmentacia w czasie testu* lub po prostu *TTA*.

TTA po prostu tworzy prognozy nie tylko na obrazach w twoim zbiorze sprawdzania poprawności, ale także tworzy prognozy dla pewnej liczby losowo rozszerzonych wersji ich (domyślnie używa oryginalnego obrazu wraz z 4 losowo rozszerzonymi wersjami). Następnie pobiera średnie prognozy z tych obrazów i wykorzystuje je. Aby użyć TTA w zestawie sprawdzania poprawności, możemy skorzystać z metody `TTA()` ucznia.

In [0]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [0]:
accuracy_np(probs, y)

Zwykle widzę o 10-20% spadku błędu w tym zestawie danych podczas korzystania z TTA w tym momencie, co jest niesamowitym wynikiem dla tak szybkiej i prostej techniki!

## Analizowanie wyników

### Macierz błędów (tablica pomyłek)

In [0]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

Popularnym sposobem analizy wyniku modelu klasyfikacyjnego jest użycie [macierzy błędów] (http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/). Scikit Learn ma wygodną funkcję, którą możemy wykorzystać do tego celu:


In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

Możemy po prostu wydrukować macierz błędów lub możemy wyświetlić widok graficzny (który jest przydatny głównie dla osób z większą liczbą kategorii).

In [0]:
mpl.style.use('default')
mpl.style.use('seaborn-ticks')

plot_confusion_matrix(cm, data.classes)

### Ponowne spojrzenie na zdjęcia

In [0]:
plot_val_with_title(most_by_correct(0, False), "Most incorrect maluchs")

In [0]:
plot_val_with_title(most_by_correct(1, False), "Most incorrect polonezes")

## Przegląd: proste kroki do wytrenowania światowej klasy klasyfikatora obrazu 

1. Włącz augmentację danych oraz cache aktywacji (precompute=True)
1. Użyj `lr_find()`, aby znaleźć najwyższe tempo uczenia się, przy którym strata (loss) jeszcze maleje
1. Wytrenuj ostatnią warstwę z prekomputowanych aktywacji przez 1-2 epoki
1. Wytrenuj ostatnią warstwę z augmentacją danych (tj. precompute = False) przez 2-3 epoki z cycle_len = 1
1. Odblokuj wszystkie warstwy
1. Ustaw dla wcześniejszych warstw 3x-10x mniejszy krok uczenia się niż dla kolejnych wyższych warstw
1. Ponownie użyj `lr_find()`
1. Trenuj całą sieć z cycle_mult = 2 aż do dopasowania

## Zrozumienie kodu naszego pierwszego modelu

Spójrzmy na kod linia po linii.

** tfms ** oznacza * transformacje *. `tfms_from_model` zajmuje się zmianą rozmiaru, kadrowaniem obrazu, początkową normalizacją (tworzenie danych za pomocą (średnia, odchylenie standardowe) (0,1)) i więcej.

In [0]:
tfms = tfms_from_model(resnet34, sz)

Potrzebujemy <b> ścieżki </b> wskazującej na zbiór danych. W tej ścieżce będziemy również przechowywać tymczasowe dane i końcowe wyniki. `ImageClassifierData.from_paths` odczytuje dane z podanej ścieżki i tworzy gotowy zestaw danych do treningu.

In [0]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)

`ConvLearner.pretrained` buduje *ucznia* który zawiera wcześniej wytrenowany model. Ostatnia warstwa modelu musi zostać zastąpiona warstwą odpowiednich wymiarów. Ten model został wytrenowany dla 1000 klas, a ostateczna warstwa przewiduje wektor o prawdopodobieństwie 1000. Model dla starych polskich samochodów musi wyprowadzić wektor dwuwymiarowy. Poniższy schemat pokazuje w przykładzie, w jaki sposób zostało to zrobione w jednym z najwcześniej odnoszących sukces CNN. Warstwa "FC8" zostanie tutaj zastąpiona nową warstwą z 2 wyjściami.

<img src="https://image.slidesharecdn.com/practicaldeeplearning-160329181459/95/practical-deep-learning-16-638.jpg" width="500">
[original image](https://image.slidesharecdn.com/practicaldeeplearning-160329181459/95/practical-deep-learning-16-638.jpg)

In [0]:
learn = ConvLearner.pretrained(resnet34, data, precompute=True)

*Parametry* są wyuczane poprzez dopasowanie modelu do danych. * Hyparameters * to kolejny rodzaj parametru, którego nie można bezpośrednio nauczyć się ze zwykłego procesu treningowego. Parametry te wyrażają właściwości "wyższego poziomu" modelu, takie jak jego złożoność lub szybkość uczenia. Dwa przykłady hiperparametrów to *tempo uczenia się* i *liczba epok*.

Podczas iteracyjnego trenowania sieci neuronowej, *wsad (batch)* lub *mini-wsad (mini-batch)* jest podzbiorem treningowych próbek używanych w jednej iteracji Stochastycznego Schodzenia Gradientowego (SGD). Epoka * * to pojedyncze przejście przez cały zestaw treningowy, który składa się z wielu iteracji SGD.

Możemy teraz *dopasować (fit)* model; to znaczy, użyj *gradientowego spadku (gradient descent)*, aby znaleźć najlepsze parametry dla w pełni połączonej warstwy, którą dodaliśmy, która może oddzielać maluchowe obrazy od polonezowych obrazów. Musimy podać dwa hipertermery: *tempo uczenia się (learning rate)* (ogólnie 1e-2 lub 1e-3 jest dobrym punktem wyjścia, przyjrzymy się temu dalej) i *liczbie epok* (możesz podać wyższy poziom liczbę i po prostu przestań trenować, gdy zauważysz, że już się nie poprawia, a następnie uruchom ponownie z liczbą odnalezionych epok).

In [0]:
learn.fit(1e-2, 1)

## Analiza wyników: strata i dokładność

Kiedy uruchamiamy `learn.fit`, drukujemy 3 wartości wydajności (patrz wyżej.) Tutaj 0.03 jest wartością **straty** w zbiorze treningowym, 0.0226 jest wartością straty w zestawie walidacyjnym, a 0,9927 jest wartością dokładność weryfikacji. Jaka jest strata? Czym jest dokładność? Dlaczego nie pokazać dokładności?

**Dokładność** to stosunek prawidłowej prognozy do całkowitej liczby prognoz.

W uczeniu maszynowym funkcja **straty** lub koszt reprezentuje cenę zapłaconą za niedokładność prognoz.

Stratę związaną z jednym przykładem w klasyfikacji binarnej daje:
 `-(y * log (p) + (1-y) * log(1-p))`
gdzie `y` jest prawdziwą etykietą `x`, a `p` jest prawdopodobieństwem przewidzianym przez nasz model, że etykieta ma wartość 1.

In [0]:
def binary_loss(y, p):
    return np.mean(-(y * np.log(p) + (1-y)*np.log(1-p)))

In [0]:
acts = np.array([1, 0, 0, 1])
preds = np.array([0.9, 0.1, 0.2, 0.8])
binary_loss(acts, preds)

Zauważ, że w naszym przykładzie zabawkowym powyżej nasza dokładność wynosi 100%, a nasza strata wynosi 0,16. Porównaj to ze stratą 0,03, którą otrzymujemy, podczas przewidywania maluchów i polonezów. 

Ćwiczenie: spróbuj zmodyfikować `preds` tak, aby uzyskać mniejszą stratę dla tego przykładu.

**Przykład:** Oto przykład, jak obliczyć stratę dla jednego przykładu binarnego problemu klasyfikacji. Załóżmy dla obrazu x z etykietą 1, a model daje prognozę 0.9. W tym przypadku strata powinna być mała, ponieważ nasz model przewiduje etykietę $1$ z wysokim prawdopodobieństwem.

`loss = -log (0.9) = 0.10`

Załóżmy teraz, że x ma etykietę 0, ale nasz model przewiduje 0.9. W tym przypadku nasza strata powinna być znacznie większa.

strata = -log (1-0,9) = 2,30


- Ćwiczenie: spójrz na inne przypadki i przekonaj się, że to ma sens.
- Ćwiczenie: w jaki sposób przepisałbyś `binary_loss` używając` if` zamiast `*` i `+`?

Dlaczego po prostu nie zwiększyć dokładności? Strata klasyfikacji binarnej jest łatwiejszą funkcją do optymalizacji.
